In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
output = model.invoke("hi")
print(output.content)

Hello! How can I help you today?


In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model="BAAI/bge-small-en")
len(embeddings.embed_query("hi"))

c:\Users\SRJ\SRJ\Work\agentic_ai\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\SRJ\SRJ\Work\agentic_ai\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SRJ\.cache\huggingface\hub\models--BAAI--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate develo

384

In [10]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
loader = DirectoryLoader("./data", glob="./*.txt", loader_cls=TextLoader)

In [12]:
docs = loader.load()

In [13]:
docs

[Document(metadata={'source': 'data\\data.txt'}, page_content="As of mid-2025, the Indian economy presents a picture of robust growth and macroeconomic stability, positioning it as one of the fastest-growing major economies in the world. Here is a snapshot of key data points and economic indicators:\n\n### Key Economic Indicators\n\n| Indicator | Value/Rate | Period | Notes |\n| :--- | :--- | :--- | :--- |\n| **GDP Growth (Real)** | **~6.5%** | FY 2024-25 | Projections for FY 2025-26 range from 6.3% to 6.8% by various agencies. |\n| **Nominal GDP** | **₹331.03 lakh crore** | FY 2024-25 | Approximately a threefold increase from ₹106.57 lakh crore in FY 2014-15. |\n| **Retail Inflation (CPI)** | **1.55%** | July 2025 | An eight-year low, primarily due to a contraction in food prices. |\n| **Fiscal Deficit** | **4.8% of GDP** | FY 2024-25 | The government has set a target of 4.4% for the financial year 2025-26. |\n| **Foreign Exchange Reserves**| **~$693.6 billion** | As of August 8, 2025